In [ ]:
# install required system dependencies
!apt-get install -y xvfb x11-utils 
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyVirtualDisplay==2.0.* \
PyOpenGL==3.1.* \
PyOpenGL-accelerate==3.1.* \
gym[box2d]==0.17.* 
!pip install pyglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 993 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 993 kB in 1s (785 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 160975 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1.

In [ ]:
!rm ./vid/*.*

rm: cannot remove './vid/*.*': No such file or directory


In [ ]:
from gym import Env, spaces
import numpy as np
import random

In [ ]:
class SteeringWheelEnv(Env):
  """
  Environment to replicate automated steering of the wheel in an environment (road)
  """
  def __init__(self):

    # The first set of action is the degree of turn
      # set of actions: turn left, hold, turn right. 0 for left turn, 1 for hold and 2 for right turn 
    # The second set of action is change in speed of the vehicle.
      # Here we will consider 0 as speed reduction and 1 as speed increase 
    self.action_space = spaces.MultiDiscrete([3,2])        
    
    # degree of turn of the road from left(negative) to right(positve)
    # self.observation_space = spaces.Box(low=np.array([-1]), high=np.array([1]))    
    
    # The environment returns 
      # (Discrete)the amount of turn required for the road ahead. Here 0 indicates left turn, 1 indicates straight road and 2 indicating right turn.
      # (Continuous)the position of the vehicle with respect to the current road lane. -1 indicating extreme left, 0 for center and 1 for extreme right
    self.observation_space = spaces.Tuple((
                                            spaces.Discrete(3),
                                            spaces.Box(low=np.array([-1]), high=np.array([1]))
                                          ))

    # intial randomized turn of the road
    self.road_turn = random.randint(0,2)
    # intial position of vehicle on the orad
    self.road_position = random.uniform(-1,1)
    
    # duration of drive
    self.drive_duration = 100   # seconds

    # hold previous action
    self.prev_action_turn = None
    # margin of error accepted for the vehicle to be considered aligned with the road (center)
    self.vehicle_road_center_threshold = 0.4


  def step(self, action):
    # decrease time step
    self.drive_duration -= 1

    action_turn, action_speed = action

    # get updated road postion of vehicle after action execution
    self.road_position = self.get_updated_road_position(self.road_position, self.road_turn, action_turn, action_speed)

    # assigning default reward to action
    reward = -1

    if self.road_turn in [0,2]:
      # when turning need to reduce speed
      if action_speed == 0:
        if self.road_turn == action_turn:
          # wheel and road turn are aligned by some margin
          if self.road_position>= -self.vehicle_road_center_threshold and self.road_position<= self.vehicle_road_center_threshold:
            reward = 1
    else:
      # for straight road
      if self.road_turn == action_turn:
          # wheel and road turn are aligned straight
          if self.road_position>= -self.vehicle_road_center_threshold and self.road_position<= self.vehicle_road_center_threshold:
            reward = 1
    
    # get next state after action
    self.road_turn, self.road_position = self.get_next_state(self.road_turn, self.road_position)

    # check if task is done     
    if self.drive_duration >0:
      task_done = False
    else:
      task_done = True

    self.prev_action_turn = action_turn
    return (self.road_turn, self.road_position), reward, task_done, {}

  
  def get_updated_road_position(self, road_position, road_turn, action_turn, action_speed):
    """
    Return the updated vehicle road position based on action
    """
    
    # NOTE: Probability is used in effect of road turn in position in order to simulate real-world scenarios(noise) 
    # like misjudged wheel turn, obstacles on road, etc
    
    update_margin = None
    # For left turning road
    if road_turn == 0:
      # For low speed
      if action_speed == 0:
        if road_turn == action_turn: # left turn aligned
          if action_turn==self.prev_action_turn:    # previous turn was also left
            update_margin = -random.uniform(0.2,0.4)  
          else:
            update_margin = -random.uniform(0,0.2)
        elif action_turn == 2:       # for vehicle turning right in a left turning road
          update_margin = random.uniform(0.2,0.4)
        else:                     # for vehicle going straight
          update_margin = random.uniform(0,0.2)
      else:
        # For higher speed
        update_margin = -random.uniform(0,0.1)

    # For right turning road
    elif road_turn == 2:
      if action_speed == 0:
        # For low speed
        if road_turn == action_turn:       # right turn aligned
          if action_turn==self.prev_action_turn:    # previous turn was also right
            update_margin = random.uniform(0.2,0.4)         
          else:
            update_margin = random.uniform(0,0.2)
        elif action_turn == 0:       # for vehicle turning left in a right turning road
          update_margin = -random.uniform(0.2,0.4)
        else:                     # for vehicle going straight
          update_margin = -random.uniform(0,0.2)
      else:
        # For high speed
        update_margin = random.uniform(0,0.1)
    
    # For straight road
    else:
      if action_turn==0:
        # vehicle turning left
        update_margin = -random.uniform(0.0,0.2)
      elif action_turn==2:
        # vehicle turning right
        update_margin = random.uniform(0.0,0.2)
      else:
        update_margin = 0.0

    if update_margin:
      road_position += update_margin
    return road_position


  def get_next_state(self, road_turn, road_position):
    """
    Get next state of environment given the current state.
    """
    next_road_turn = None
    if road_turn == 0:
      # cannot go from left to right directly
      next_road_turn = random.randint(0, 1)
    elif road_turn == 1:
      next_road_turn = random.randint(0,2)
    else:
      # cannot go from right to left directly
      next_road_turn = random.randint(1,2)

    if next_road_turn == 0:
      # For left
      road_position += random.uniform(0.0, 0.2)
    elif next_road_turn == 2:
      road_position -= random.uniform(0.0, 0.2)
    else:
      road_position = road_position   # no change for straight road
    return next_road_turn, road_position


  def reset(self):
    """
    Reset environment states
    """
    # intial randomized turn of the road
    self.road_state = random.randint(0,2)
    # intial position of vehicle on the orad
    self.road_position = random.uniform(-1,1)
    
    # reset duration of drive
    self.drive_duration = 100   # seconds
    return self.road_state, self.road_position


In [ ]:
env = SteeringWheelEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
print('Sample obsercation from environment:',env.observation_space.sample())
# The first param is the turn of road representation and second is the position of vehicle on the road

Sample obsercation from environment: (1, array([0.07008331], dtype=float32))


In [ ]:
# testing random actions on the environment
for epoch in range(10):
  obs = env.reset()
  done = False
  total_reward = 0
  while not done:
    obs, reward, done, i = env.step(env.action_space.sample())
    total_reward += reward
  print('Test: {}, Total Reward Collected: {}' .format(epoch, total_reward))

Test: 0, Total Reward Collected: -82
Test: 1, Total Reward Collected: -76
Test: 2, Total Reward Collected: -98
Test: 3, Total Reward Collected: -62
Test: 4, Total Reward Collected: -96
Test: 5, Total Reward Collected: -100
Test: 6, Total Reward Collected: -100
Test: 7, Total Reward Collected: -100
Test: 8, Total Reward Collected: -98
Test: 9, Total Reward Collected: -98


In [ ]:
# Now we aim to create an optimal policy based on the observation
def policy(obs):
  # here observation is the road condition and position of vehicle in road
  road_turn, vehicle_road_pos = obs
  turn = 1    # default action is to go straight
  speed = 1   # default speed action

  if road_turn == 1:
    # going straight
    turn = 1
  elif road_turn == 0:
    # For left turning road
    speed = 0
    if vehicle_road_pos>0.2:
      turn = 0
  else:
    # For right turning road
    if vehicle_road_pos<-0.2:
      turn = 2

  return turn, speed

In [ ]:
# testing implemented agent's policy on the environment
o = env.reset()
done = False
total_reward = 0
while not done:
  action = policy(o)
  o, reward, done, i = env.step(action)
  total_reward += reward
print('Total Reward Collected: {}' .format(total_reward))

Total Reward Collected: 18
